# Imports, Functions & Testing

In [1]:
import pandas as pd
import numpy as np
import requests
import re
import pickle
from bs4 import BeautifulSoup

In [2]:
def gen_election_cleaner(df, verbose=1):
    import pandas as pd
    
    df_cln = df.copy()
    
    if verbose in [1,2]:
        ## Printing startpoint
        print('O.G. Copy:')
        print('---'*20)
        display(df_cln)

    ## Setting columns manually + Q.C.
    df_cln.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
    if verbose == 2:
        print('\nNew Column Headers:')
        print('---'*20)
        display(df_cln)

    ## Dropping irrelevant information + Q.C.
    df_cln.drop(index=[0, 6, 8], inplace=True)
    if verbose == 2:
        print('\nDropping Rows:')
        print('---'*20)
        display(df_cln)

    ## Flip and place Turnout as its own feature + Q.C.
    for idx, v, k in zip(df_cln.index, df_cln['Null'], df_cln['Null'].isna()):
        if not k:
            df_cln[v] = df_cln['Party'][idx]

    ## Removing the row with turnout info/column with nulls + Q.C.
    df_cln.drop(columns='Null', index=7, inplace=True)
    if verbose in [1,2]:
        print('\nFinal Version:')
        print('---'*20)
        display(df_cln)

    return df_cln

In [3]:
def wiki_senate_scraper():
    import pandas as pd
    import numpy as np
    import requests
    from bs4 import BeautifulSoup
    
    ## Starting page + Q.C. of response
    start_url = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
    start_resp = requests.get(start_url)
    print(f'Starting Response: {start_resp}')
    
    ## Creating soup + pulling all links for senate elections after 17th amendment
    start_soup = BeautifulSoup(start_resp.text, 'html.parser')
    start_links = start_soup.findAll('a')
    start_sen_links = start_links[78:134] ## Previously located
    
    ## Base of url for all senate election pages
    base_url = 'https://en.wikipedia.org'
    
    ## Loop same process for all links + storage
    yr_dfs = {}
    yr_tocs = {}
    count = 0
    for link in start_sen_links:

        ## Q.C during execute
        count += 1
        if count in [25, 50]:
            print('Checkpoint! (25 loops)')
        
        ## Collecting strings for use
        end_url = link.get('href')
        year = link.get_text()
        full_url = base_url + end_url
        
        ## Making soup + collecting all tables
        link_resp = requests.get(full_url)
        link_soup = BeautifulSoup(link_resp.text, 'html.parser')
        link_tables = link_soup.findAll('table', attrs={'class': ['wikitable', 'infobox vevent', 'infobox']})
        
        ## Collecting list of states with elections in each year
        link_toc = link_soup.find('div', attrs={'id':'toc'})
        link_toc = link_toc.findAll('a', href=is_state)
        toc_list = [tag.get('href').replace('#', '') for tag in link_toc]
        
        ## Converting to dataframe + storage
        elect_df = pd.read_html(str(link_tables))
        yr_dfs[year] = elect_df
        yr_tocs[year] = toc_list
    
    print(f'Total pages scraped: {count}')
    return yr_dfs, yr_tocs

## https://www.crummy.com/software/BeautifulSoup/bs4/doc/#kinds-of-filters

In [4]:
def is_state(href):
    states_list = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 
                   'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
                   'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
                   'Kentucky', 'Louisiana', 'Maine', 'Maryland',
                   'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
                   'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New_Hampshire',
                   'New_Jersey', 'New_Mexico', 'New_York', 'North_Carolina',
                   'North_Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
                   'Rhode_Island', 'South_Carolina', 'South_Dakota',
                   'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia',
                   'Washington', 'West_Virginia', 'Wisconsin', 'Wyoming']
    for state in states_list:
        if state in href:
            return href

In [5]:
def election_collector(dict_tables, dict_lists):
    
    ## Requirement of proper dictionaries
    if dict_tables.keys() != dict_lists.keys():
        print('**WARNING**')
        print('Keys do not match in dictionaries passed. Adjust and try again.')
        return '***'*10
    
    ## Containers for results
    coll_elects = {}
    
    ## List creation for looping through dicts
    yr_list = list(dict_lists.keys())
    
    ## Looping + storage
    for year in yr_list:
        print('***'*10)
        print(f"Collecting {year}'s elections...")
        yr_tables = dict_tables[year]
        yr_toc = dict_lists[year]
        
        ## Creating containers to further separate data
        yr_sum_ldrs = yr_tables[2]
        most_tables = yr_tables[3:]
        yr_summary = []
        yr_states = []
        count = 0
        
        for i, df in enumerate(most_tables):
            if count < 1:
                if df.shape[1] is 6 and most_tables[i+1].shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
                    yr_summary.append(most_tables[i+1])
                    continue
                elif df.shape[1] is 6:
                    count += 1
                    yr_summary.append(most_tables[i])
            
            if df.shape[1] in [4,5,6] and df.shape[0] < 15:
                yr_states.append(most_tables[i])
        
        coll_elects[year] = [yr_sum_ldrs, yr_summary, yr_states]
        
    print('\n')
    print('---'*20)
    print(f'Collection Complete: Data is for {yr_list[0]} - {yr_list[-1]}')
    print('---'*20)
    return coll_elects

# Initial Data

> [Link to Dataset](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/PEJ5QU)

In [6]:
df = pd.read_csv('dataverse_files/1976-2018-senate.csv', encoding='latin-1')
df.head()

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421 entries, 0 to 3420
Data columns (total 18 columns):
year              3421 non-null int64
state             3421 non-null object
state_po          3421 non-null object
state_fips        3421 non-null int64
state_cen         3421 non-null int64
state_ic          3421 non-null int64
office            3421 non-null object
district          3421 non-null object
stage             3421 non-null object
special           3421 non-null bool
candidate         3011 non-null object
party             2858 non-null object
writein           3421 non-null bool
mode              3421 non-null object
candidatevotes    3421 non-null int64
totalvotes        3421 non-null int64
unofficial        3421 non-null bool
version           3420 non-null float64
dtypes: bool(3), float64(1), int64(6), object(8)
memory usage: 411.0+ KB


In [8]:
df[(df['state_po'] == 'AZ') & (df['year'] == 1976)]

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version
0,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Sam Steiger,republican,False,total,321236,741210,False,20171011.0
1,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Wm. Mathews Feighan,independent,False,total,1565,741210,False,20171011.0
2,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Dennis DeConcini,democrat,False,total,400334,741210,False,20171011.0
3,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Allan Norwitz,libertarian,False,total,7310,741210,False,20171011.0
4,1976,Arizona,AZ,4,86,61,US Senate,statewide,gen,False,Bob Field,independent,False,total,10765,741210,False,20171011.0


In [9]:
# len(df[df['candidate'].isna()])

# df['writein'].value_counts()

# for col in df:
#     display(df[col].value_counts())

# pd.Series.value_counts()

# Additional Data

Current set of features includes:
* Year of election
* State (encoded)
* 

> [Link to Brookings](https://www.brookings.edu/multi-chapter-report/vital-statistics-on-congress/)

> [Link to BallotPedia](https://ballotpedia.org/Legislative_Branch)

> [Link to WikiPedia](https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections)

> [Link to Wiki-Category (U.S. Senator)](https://commons.wikimedia.org/wiki/Category:Senators_of_the_United_States)

## Testing tables 'infobox vevent'

In [10]:
# url3 = 'https://en.wikipedia.org/wiki/1978_United_States_Senate_elections'
# response3 = requests.get(url3)
# print(response3)

# soup3 = BeautifulSoup(response3.text, 'html.parser')
# wiki_tables3 = soup3.findAll('table', attrs={'class': ['wikitable', 'infobox vevent']})

# print(type(wiki_tables3))

# test_df3 = pd.read_html(str(wiki_tables3))

# tester = test_df3[0]

In [11]:
# https://stackoverflow.com/questions/27156278/index-pandas-dataframe-by-column-numbers-when-column-names-are-integers
# tester.columns = ['a', 'b', 'c']
# display(tester.iloc[6:18,0:3])

# display(test_df3[2])

In [12]:
# url = 'https://en.wikipedia.org/wiki/1976_United_States_Senate_elections'
# response = requests.get(url)
# print(response)

# soup = BeautifulSoup(response.text, 'html.parser')
# wiki_tables = soup.findAll('table', attrs={'class': 'wikitable'})

# type(wiki_tables)

In [13]:
# test_df = pd.read_html(str(wiki_tables))
# display(test_df[0])

# len(test_df)

## Pulling state election tables

In [14]:
# ## Pulling Arizona gen election results
# tester = test_df[5].copy()
# print('O.G. Copy:')
# print('---'*20)
# display(tester)

# ## Setting columns manually + Q.C.
# tester.columns = ['Null', 'Party', 'Candidate', 'Votes', '%of_Vote']
# print('\nNew Column Headers:')
# print('---'*20)
# display(tester)

# ## Dropping irrelevant information + Q.C.
# tester.drop(index=[0, 6, 8], inplace=True)
# print('\nDropping Rows:')
# print('---'*20)
# display(tester)

# ## Flip and place Turnout as its own feature + Q.C.
# for idx, v, k in zip(tester.index, tester.Null, tester.Null.isna()):
#     if not k:
#         #print(idx, v)
#         tester[v] = tester.Party[idx]

# ## Removing the row with turnout info/column with nulls + Q.C.
# try:
#     tester.drop(columns='Null', index=7, inplace=True)
#     print('\nFinal Version:')
#     print('---'*20)
#     display(tester)
# except:
#     print('Something went wrong!')

In [15]:
# type(tester['Votes'][5]) # str

In [16]:
# tester2 = gen_election_cleaner(test_df[6], verbose=2)

In [17]:
# test_df[-1]

In [18]:
# url2 = 'https://en.wikipedia.org/wiki/List_of_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [19]:
# url2 = 'https://en.wikipedia.org/wiki/2018_United_States_Senate_elections'
# response2 = requests.get(url2)
# print(response2)

In [20]:
# soup2 = BeautifulSoup(response2.text, 'html.parser')
# wiki_toc = soup2.find('div', attrs={'id':'toc'})
# wiki_toc = wiki_toc.findAll('a', href=is_state)
# display(wiki_toc[0:5])
# jiji = wiki_toc[0:5]

# jiji = [tag.get('href').replace('#', '') for tag in jiji]
# jiji = jiji
# jiji

## Working

In [21]:
# test = wiki_links[78:134]
# beeb = test[0].get_text()
# geeb = 'https://en.wikipedia.org'
# geeb + beeb

In [22]:
# display(elect_yr_dfs.keys())
# display(elect_yr_tocs.keys())
# display(len(elect_yr_dfs) == len(elect_yr_tocs))
# elect_yr_dfs.keys() != elect_yr_tocs.keys()
# list(elect_yr_dfs.keys())

In [23]:
# elect_yr_dfs['1918']

In [24]:
# elect_yr_tocs['1918']

In [25]:
# year = '1924'

# for i, df in enumerate(elect_yr_dfs[year]):
#     print(i, df.shape)

# display(elect_yr_dfs[year][10])
# elect_yr_dfs[year][10].shape[1] is 6

In [26]:
# koko = {v:elect_yr_dfs[v] for v in ['1918', '1924', '1976', '2016', '2018']}
# lplp = {v:elect_yr_tocs[v] for v in ['1918', '1924', '1976', '2016', '2018']}

In [27]:
# year = '2016'

# for v in ftft[year]:
#     print(len(v))
    
# ftft[year][1]

In [86]:
# yuyu = test_res[-1].append(test_res[-2], ignore_index=True)
# yuyu

In [59]:
# yuyu = test_res[-1].iloc[1,:] == test_res[-2].iloc[1,:]
# yuyu.sum()

In [87]:
# len(test_res)

## Getting it together

In [34]:
# elect_yr_tables, elect_yr_tocs = wiki_senate_scraper()

# election_collection_raw = election_collector(elect_yr_tables, elect_yr_tocs)

******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
******************************
********

In [37]:
# with open('election_collection_raw.pickle', 'wb') as f:
#     pickle.dump(election_collection_raw, f)
#     f.close()

In [6]:
with open('election_collection_raw.pickle', 'rb') as f:
    election_collection_raw = pickle.load(f)
    f.close()

In [22]:
year = '2006'

display(election_collection_raw.keys())
print('\n')
for grp in election_collection_raw[year]:
    print(len(grp))
print('\n')
display(election_collection_raw[year][0])
display(election_collection_raw[year][1])
display(election_collection_raw[year][2])

dict_keys(['1914', '1916', '1918', '1920', '1922', '1924', '1926', '1928', '1930', '1932', '1934', '1936', '1938', '1940', '1942', '1944', '1946', '1948', '1950', '1952', '1954', '1956', '1958', '1960', '1962', '1964', '1966', '1968', '1970', '1972', '1974', '1976', '1978', '1980', '1982', '1984', '1986', '1988', '1990', '1992', '1994', '1996', '1998', '2000', '2002', '2004', '2006', '2008', '2010', '2012', '2014', '2016', '2018', '2020', '2022', '2024'])



25
2
103




,0,1,2
0,NaN,NaN,NaN
1,Leader,Harry Reid,Bill Frist(retired)
2,Party,Democratic,Republican
3,Leader since,"January 3, 2005","January 3, 2003"
4,Leader's seat,Nevada,Tennessee
5,Seats before,44,55
6,Seats after,49,49
7,Seat change,5,6
8,Popular vote,32344708,25437934
9,Percentage,53.2%,41.8%


[                0               1                       2                3  \
 0           State       Incumbent         Crystal Ball[2]    Rothenberg[3]   
 1         Arizona         Kyl (R)                 Leans R          Leans R   
 2      California   Feinstein (D)                  Safe D           Safe D   
 3     Connecticut   Lieberman (D)         Leans Lieberman           Safe D   
 4        Delaware      Carper (D)                  Safe D           Safe D   
 5         Florida      Nelson (D)                  Safe D           Safe D   
 6          Hawaii       Akaka (D)                  Safe D           Safe D   
 7         Indiana       Lugar (R)                  Safe R           Safe R   
 8           Maine       Snowe (R)                  Safe R           Safe R   
 9        Maryland  (Sarbanes) (D)                 Leans D          Leans D   
 10  Massachusetts     Kennedy (D)                  Safe D           Safe D   
 11       Michigan    Stabenow (D)                Li

[              0           1             2   3
 0           NaN         NaN           NaN NaN
 1       Nominee     Jon Kyl  Jim Pederson NaN
 2         Party  Republican    Democratic NaN
 3  Popular vote      814398        664141 NaN
 4    Percentage       53.3%         43.5% NaN,
              0           1                    2       3      4
 0        Party   Candidate                Votes       %    NaN
 1          NaN  Republican  Jon Kyl (Incumbent)  297636  99.5%
 2          NaN  Republican            Write-ins     155  0.05%
 3  Total votes      297791              100.00%     NaN    NaN,
              0           1             2       3        4
 0        Party   Candidate         Votes       %      NaN
 1          NaN  Democratic  Jim Pederson  214455  100.00%
 2  Total votes      214455       100.00%     NaN      NaN,
              0            1             2     3        4
 0        Party    Candidate         Votes     %      NaN
 1          NaN  Libertarian  Richard Mack 

In [ ]:
test_list = [election_collection_raw[year][2][8], election_collection_raw[year][2][3]]
for u in test_list:
    
    display(u.iloc[:,2])

In [12]:
# election_collection_raw[year][1][1]
for i, df in enumerate(election_collection_raw[year][2]):
    display(df)

,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Nominee,Jeff Sessions,Vivian Davis Figures,NaN
2,Party,Republican,Democratic,NaN
3,Popular vote,1305383,752391,NaN
4,Percentage,63.36%,36.52%,NaN


,0,1,2,3,4
0,Party,Candidate,Votes,%,NaN
1,NaN,Republican,Jeff Sessions,199690,92.27%
2,NaN,Republican,Earl Mack Gavin,16718,7.73%
3,Total votes,216408,100.00%,NaN,NaN


,0,1,2,3,4
0,Party,Candidate,Votes,%,NaN
1,NaN,Democratic,Vivian Davis Figures,112074,63.72%
2,NaN,Democratic,Johnny Swanson,38757,22.03%
3,NaN,Democratic,Mark Townsend,25058,14.25%
4,Total votes,175889,100.00%,NaN,NaN


,0,1,2,3,4
0,Party,Candidate,Votes,%,NaN
1,NaN,Republican,Jeff Sessions (Incumbent),1305383,63.36%
2,NaN,Democratic,Vivian Figures,752391,36.52%
3,NaN,Write-in,Write-ins,2417,0.12%
4,Invalid or blank votes,NaN,NaN,NaN,NaN
5,Total votes,2060191,100.00%,NaN,NaN
6,Turnout,NaN,NaN,NaN,NaN
7,NaN,Republican hold,NaN,NaN,NaN


,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Nominee,Mark Begich,Ted Stevens,NaN
2,Party,Democratic,Republican,NaN
3,Popular vote,151767,147814,NaN
4,Percentage,47.8%,46.5%,NaN


...

,0,1,2,3,4,5
0,Party,Candidate,Votes,%,±,NaN
1,NaN,Republican,Mike Enzi,189046,75.63%,+2.68%
2,NaN,Democratic,Chris Rothfuss,60631,24.26%,-2.79%
3,NaN,Write-ins,NaN,269,0.11%,NaN
4,Majority,128415,51.38%,+5.47%,NaN,NaN
5,Turnout,249946,NaN,NaN,NaN,NaN
6,NaN,Republican hold,Swing,NaN,NaN,NaN


,0,1,2,3
0,NaN,NaN,NaN,NaN
1,Nominee,John Barrasso,Nick Carter,NaN
2,Party,Republican,Democratic,NaN
3,Popular vote,183063,66202,NaN
4,Percentage,73.4%,26.5%,NaN


,0,1,2,3,4
0,Party,Candidate,Votes,%,NaN
1,NaN,Democratic,Nick Carter,12316,50.68%
2,NaN,Democratic,Keith Goodenough,11984,49.32%
3,Total votes,22799,100.00%,NaN,NaN


,0,1,2,3,4
0,Party,Candidate,Votes,%,NaN
1,NaN,Republican,John Barrasso (Incumbent),68194,100.00%
2,Total votes,68194,100.00%,NaN,NaN


,0,1,2,3,4,5
0,Party,Candidate,Votes,%,±,NaN
1,NaN,Republican,John Barrasso,183063,73.35%,+3.37%
2,NaN,Democratic,Nick Carter,66202,26.53%,-3.33%
3,NaN,None,Write-ins,293,0.12%,NaN
4,Majority,116861,46.83%,+6.70%,NaN,NaN
5,Turnout,249558,NaN,NaN,NaN,NaN
6,NaN,Republican hold,Swing,NaN,NaN,NaN


In [15]:
print(election_collection_raw[year][1][0].shape)
print(election_collection_raw[year][1][1].shape)

(3, 6)
(35, 6)


In [24]:
for u in election_collection_raw[year][1][0].iloc[:,4]:
    if isinstance(u, str) and 'elected' in u:
        print('faslidfjasedfhaedfng')
        
for u in election_collection_raw[year][1][1].iloc[:,4]:
    if isinstance(u, str) and 'elected' in u:
        print('faslidfjasedfhaedfng')

faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng
faslidfjasedfhaedfng


In [25]:
test_list = []
test_list.append(election_collection_raw['1918'])
test_list.append(election_collection_raw['1920'])
test_list.append(election_collection_raw['1976'])
test_list.append(election_collection_raw['2006'])
test_list.append(election_collection_raw['2008'])

In [83]:
test_res = []
for coll in test_list:
    holder = []
    for table in coll[1]:
        has_elected = False
        
        for u in table.iloc[:,4]:
            if isinstance(u, str) and 'elected' in u:
                has_elected = True
        if has_elected == True:
            holder.append(table)
    
        print(len(holder))
    if len(holder) > 1:
        is_match = holder[0].iloc[1,:] == holder[1].iloc[1,:]
    
        if is_match.sum() == 3:
            fin_tab = holder[1].append(holder[0], ignore_index=True)
            test_res.append(fin_tab)
    else:
        test_res.extend(holder)

1
2
1
2
1
0
1
1
2


In [84]:
test_res[-1]

,0,1,2,3,4,5
0,State(linked tosummaries below),Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoral history,NaN,NaN,NaN
2,Alabama,Jeff Sessions,Republican,19962002,Incumbent re-elected.,Jeff Sessions (Republican) 63.4% Vivian Davis ...
3,Alaska,Ted Stevens,Republican,1968 (Appointed)1970 (Special)1972197819841990...,Incumbent lost re-election.New member elected....,Mark Begich (Democratic) 47.8% Ted Stevens (Re...
4,Arkansas,Mark Pryor,Democratic,2002,Incumbent re-elected.,Mark Pryor (Democratic) 79.5% Rebekah Kennedy ...
5,Colorado,Wayne Allard,Republican,19962002,Incumbent retired.New member elected.Democrati...,Mark Udall (Democratic) 52.8% Bob Schaffer (Re...
6,Delaware,Joe Biden,Democratic,197219781984199019962002,Incumbent re-elected.,Joe Biden (Democratic) 64.7% Christine O'Donne...
7,Georgia,Saxby Chambliss,Republican,2002,Incumbent re-elected.,Saxby Chambliss (Republican) 49.8% Jim Martin ...
8,Idaho,Larry Craig,Republican,199019962002,Incumbent retired.New member elected.Republica...,Jim Risch (Republican) 57.7% Larry LaRocco (De...
9,Illinois,Dick Durbin,Democratic,19962002,Incumbent re-elected.,Dick Durbin (Democratic) 67.8% Steve Sauerberg...


In [85]:
test_res[-2]

,0,1,2,3,4,5
0,State(linked tosections below),Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoral history,NaN,NaN,NaN
2,Arizona,Jon Kyl,Republican,19942000,Incumbent re-elected.,Jon Kyl (Republican) 53.3% Jim Pederson (Democ...
3,California,Dianne Feinstein,Democratic,1992 (Special)19942000,Incumbent re-elected.,Dianne Feinstein (Democratic) 59.4% Dick Mount...
4,Connecticut,Joe Lieberman,Democratic,198819942000,"Incumbent lost renomination, but re-elected.Co...",Joe Lieberman (Connecticut for Lieberman) 49.7...
5,Delaware,Tom Carper,Democratic,2000,Incumbent re-elected.,Tom Carper (Democratic) 67.1% Jan Ting (Republ...
6,Florida,Bill Nelson,Democratic,2000,Incumbent re-elected.,Bill Nelson (Democratic) 60.3% Katherine Harri...
7,Hawaii,Daniel Akaka,Democratic,1990 (Appointed)1990 (Special)19942000,Incumbent re-elected.,Daniel Akaka (Democratic) 61.4% Cynthia Thiele...
8,Indiana,Richard Lugar,Republican,19761982198819942000,Incumbent re-elected.,Richard Lugar (Republican) 87.3% Steve Osborn ...
9,Maine,Olympia Snowe,Republican,19942000,Incumbent re-elected.,Olympia Snowe (Republican) 74.4% Jean Hay Brig...


In [88]:
test_res[-3]

,0,1,2,3,4,5
0,State,Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoral history,NaN,NaN,NaN
2,Arizona,Paul Fannin,Republican,19641970,Incumbent retired.New senator elected.Democrat...,Dennis DeConcini (Democratic) 54.0% Sam Steige...
3,California,John V. Tunney,Democratic,19701971 (Appointed),Incumbent lost re-election.New senator elected...,S. I. Hayakawa (Republican) 50.2% John V. Tunn...
4,Connecticut,Lowell Weicker,Republican,1970,Incumbent re-elected.,Lowell Weicker (Republican) 57.7% Gloria Schaf...
5,Delaware,William Roth,Republican,19701971 (Appointed),Incumbent re-elected.,William Roth (Republican) 55.8% Thomas C. Malo...
6,Florida,Lawton Chiles,Democratic,1970,Incumbent re-elected.,Lawton Chiles (Democratic) 63.0% John Grady (R...
7,Hawaii,Hiram Fong,Republican,195919641970,Incumbent retired.New senator elected.Democrat...,Spark Matsunaga (Democratic) 53.7% William F. ...
8,Indiana,Vance Hartke,Democratic,195819641970,Incumbent lost re-election.New senator elected...,Richard Lugar (Republican) 58.8% Vance Hartke ...
9,Maine,Edmund Muskie,Democratic,195819641970,Incumbent re-elected.,Edmund Muskie (Democratic) 60.2% Robert A. G. ...


In [89]:
test_res[-4]

,0,1,2,3,4,5
0,State,Incumbent,Results,Candidates,NaN,NaN
1,Senator,Party,Electoral history,NaN,NaN,NaN
2,Alabama,Oscar W. Underwood,Democratic,1914,Incumbent re-elected.,"Oscar W. Underwood (Democratic), 66.% L. H. Re..."
3,Arizona,Marcus A. Smith,Democratic,19121914,Incumbent lost re-election.New senator elected...,"Ralph H. Cameron (Republican), 55.2% Marcus A...."
4,Arkansas,William F. Kirby,Democratic,1916 (Special),Incumbent lost renomination.New senator electe...,"Thaddeus H. Caraway (Democratic), 65.9% Charle..."
5,California,James D. Phelan,Democratic,1914,Incumbent lost re-election.New senator elected...,"Samuel M. Shortridge (Republican), 49.0% James..."
6,Colorado,Charles S. Thomas,Democratic,1913 (Special)1914,Incumbent retired.New senator elected.Republic...,"Samuel D. Nicholson (Republican), 54.5% Tully ..."
7,Connecticut,Frank B. Brandegee,Republican,1905 (Special)19091914,Incumbent re-elected.,"Frank B. Brandegee (Republican), 59.3% Augusti..."
8,Florida,Duncan U. Fletcher,Democratic,1909 (Appointed)1909 (Special)1914,Incumbent re-elected.,"Duncan U. Fletcher (Democratic), 69.5% John M...."
9,Georgia,Hoke Smith,Democratic,1911 (Special)1914,Incumbent lost renomination.New senator electe...,"Thomas E. Watson (Democratic), 94.9% Harvey S...."


* Create a table with names as index and state as values .loc name of candidates to get state value
* Use table of each year's general summary to then pull the general elections out of whole list (index 2 in colleciton) based upon names in table + non-uniform party column
    * Need to merge (if special election) tables in collection index 1

* 1924 = 10
* 1926 = 12
* 1928 = 10
* Pull only tables with `shape[1]` of 5 or 6.
* Table 2 is always summary of senate leaders

Why after the 17th amendment?? [Link](https://en.wikipedia.org/wiki/Seventeenth_Amendment_to_the_United_States_Constitution)

For 1920 forward, proceed like 1976. Prior to this, will need to use first table with general info.

* **Clean up NaN value table from top of pages** -- Use index 2 to grab  nested table from `'infobox vevent'`.
* Take list of states of top tables and map to dfs
* Get demographic info wikidata
* eda time series of state party (heatmap)
* NLP on names, etc.
* [link for slider viz](https://medium.com/@sjacks/the-journey-to-an-attractive-visualization-bac019506a49)